In [1]:
# Utility definitions
import subprocess
import os
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)
import re
import shutil
import os

bashPath = os.path.join(os.environ['PROGRAMFILES'], 'Git\\bin\\sh.exe')
if not os.path.exists(bashPath):
    raise Exception('Cannot find "{}".'.format(bashPath))

    
def runBash(cmd):
    subprocess.Popen([bashPath, 
                      '--login',
                      '-i', 
                      '-c', 
                      '{} ; read -n 1 -s -r -p "Press any key to exit..."'.format(cmd)],
                     creationflags=subprocess.CREATE_NEW_CONSOLE)
    
    
@register_cell_magic
def bash(line, cell=None):
    cmd = cell
    subprocess.Popen([bashPath, '--login','-i', '-c', '{} ; read -n 1 -s -r -p "Press any key to exit..."'.format(cmd)], 
                     creationflags=subprocess.CREATE_NEW_CONSOLE)
    
    
def replaceInFile(srcFile, destFile, replace):
    with open(srcFile, 'r') as f:
        text = f.read()
    
    for k, v in replace.items():
        patt = k.replace('.', '\\.')
        patt = re.compile(patt)
        match = patt.search(text)
        if match is None:
            raise Exception("Can't find '{0}' in config file.".format(k))

        text = re.sub(patt, v, text)
        print(match.group(0), "=>", v)

    with open(destFile, 'w') as f:
        f.write(text)

In [16]:
%%bash
IMAGE_SIZE=224
ARCHITECTURE="mobilenet_0.50_${IMAGE_SIZE}"

# 'Bottleneck': last hidden layer
# The script speeds things up to cache these bottleneck values on disk
#
python tensorflow/tensorflow/examples/image_retraining/retrain.py \
    --bottleneck_dir=tf_files/bottlenecks \
    --how_many_training_steps=500 \
    --model_dir=tf_files/models/ \
    --summaries_dir=tf_files/training_summaries/"${ARCHITECTURE}" \
    --output_graph=tf_files/retrained_graph.pb \
    --output_labels=tf_files/retrained_labels.txt \
    --architecture="${ARCHITECTURE}" \
    --image_dir=tf_files/Marcel-Train

In [23]:
%%bash
python tensorflow-for-poets-2/scripts/label_image.py \
    --graph=tf_files/retrained_graph.pb  \
    --image=C2.jpg